# SBTi-Finance Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the SBTi-Finance Tool. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portoflio temperature score. It also shows you how to calculate the portfolio coverage. 

Please see the [methodology, guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details. 


See 1_analysis_example for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

### Install the SBTi Python module
This is only required if you have not already installed the module.

In [1]:
!pip install SBTi

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple
  Cloning git://github.com/OFBDABV/SBTi to c:\users\jorisc\appdata\local\temp\pip-req-build-x4zhg4tq
  Created wheel for SBTi: filename=SBTi-0.5-py3-none-any.whl size=289058 sha256=8931e67a3964b762f81f435300402c24318c2d0728ccce12113c06afb7534663
  Stored in directory: C:\Users\jorisc\AppData\Local\Temp\pip-ephem-wheel-cache-1a3qyl_i\wheels\9b\39\b9\ad6d643283a5dc18ada60798167ce0057729af6a4640ad4a16
Successfully built SBTi


In [16]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://ofbdabv.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental. 


In [17]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/utils.py", "utils.py")
    
try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations
except:  # Import statement when run locally
    from examples.utils import  print_aggregations

##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [18]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. For all available dataproviders check the implementation [here](https://github.com/OFBDABV/SBTi/tree/master/SBTi/data)




In [19]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://ofbdabv.github.io/SBTi/Legends.html#) for details on data requirements. 

In [20]:
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

In [21]:
df_portfolio.head(5)

,company_name,company_id,company_isin,weights,investment_value,engagement_target
0,"Advanced Micro Devices, Inc",US0079031078,US0079031078,6.36%,35000000,True
1,Adobe Systems Inc.,US00724F1012,US00724F1012,1.82%,10000000,NaN
2,Capgemini Group,FR0000125338,FR0000125338,1.82%,10000000,NaN
3,"Cisco Systems, Inc.",US17275R1023,US17275R1023,1.82%,10000000,NaN
4,Coca-Cola HBC AG,CH0198251305,CH0198251305,1.82%,10000000,NaN


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://ofbdabv.github.io/SBTi/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [22]:
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [23]:
temperature_score = TemperatureScore(
    time_frames=list(SBTi.interfaces.ETimeFrames), 
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],
    aggregation_method=PortfolioAggregationMethod.WATS
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

In [24]:
amended_portfolio.head(10)

,company_id,time_frame,scope,target_type,intensity_metric,coverage_s1,coverage_s2,coverage_s3,reduction_ambition,base_year,...,annual_reduction_rate,slope,samplesize,model,variable,param,intercept,r2,temperature_score,temperature_results
0,US0079031078,SHORT,S1S2,absolute,nan,1.0,1.0,NaN,0.30,2015.0,...,0.060000,slope5,128.0,4.0,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,1.30,0.000000
1,US0079031078,SHORT,S3,absolute,nan,NaN,NaN,0.95,0.97,2014.0,...,0.121250,slope5,128.0,4.0,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,0.00,0.000000
2,US0079031078,SHORT,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope5,NaN,NaN,NaN,NaN,NaN,NaN,0.35,0.000000
3,US0079031078,MID,S1S2,Absolute,nan,1.0,1.0,NaN,0.30,2018.0,...,0.042857,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,1.36,0.000000
4,US0079031078,MID,S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.20,1.000000
5,US0079031078,MID,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope15,NaN,NaN,NaN,NaN,NaN,NaN,2.70,0.727326
6,US0079031078,LONG,S1S2,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope30,128.0,4.0,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
7,US0079031078,LONG,S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope30,128.0,4.0,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
8,US0079031078,LONG,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope30,NaN,NaN,NaN,NaN,NaN,NaN,3.20,1.000000
9,US00724F1012,SHORT,S1S2,absolute,None,NaN,NaN,NaN,NaN,NaN,...,NaN,slope5,128.0,4.0,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,3.20,1.000000


## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. For more details, please refer to notebook 4 ("4_portfolio_aggregations.ipynb") and the methodology section **[replace with correct section]**. The termperature scores are calculated per time-frame/scope combination.


In [25]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

Timeframe  Scope      Temp score
short      S1S2       2.54
short      S3         2.93
short      S1S2S3     2.83
mid        S1S2       2.63
mid        S3         2.98
mid        S1S2S3     2.88
long       S1S2       3.16
long       S3         3.23
long       S1S2S3     3.21


## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.
o calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. In this example we use the "Weighted Average Temperature Score" (WATS). For more details on aggregation methods and the portfolio coverage method, please refer to the [methodoloogy document](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf) sections 3.2 and also turn to notebook 4 ("4_portfolio_aggregations.ipynb") for more aggregation examples.

In [26]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
round(coverage * 100) / 100

35.45